In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import proxsuite

# **2. Quadratic Programming via a Primal-Dual Augmented Lagrangian Method**

Implement the Primal-Dual ALM (you are not obliged to follow Algo. 1 exactly; it is merely given as a guideline). You are allowed to use only the native Python and the numpy library.

In [ ]:
# Generate a random non-trivial quadratic program.
n = 4 # number of dimensions of the optimization variables (x)
p = 1 # number of equality constraints
m = 2 # number of inequality constraints

# Random Cost
Q = np.random.randn(n, n)
Q = Q.T @ Q # We need to make sure that Q is positive definite
q = np.random.randn(n)

# Random equality constraints
A = np.random.randn(p, n)
b = np.random.randn(p)

# Random inequality constraints
C = np.random.randn(m, n)
d = C @ np.random.randn(n)

In [ ]:
# Let's create the solver
qp_dim = Q.shape[0]
qp_dim_eq = A.shape[0]
qp_dim_in = C.shape[0]
qp = proxsuite.proxqp.dense.QP(qp_dim, qp_dim_eq, qp_dim_in)

# initialize the model of the problem to solve
qp.init(Q, q, A, b, C, None, d)
qp.solve()

In [ ]:
# Optimization function
def f(x):
    return (1/2) * x.T @ Q @ x + q.T @ x

# Projection onto the nonnegative orthant
def projection_nonnegative(a):
    return np.maximum(a, 0)

# Residuals and Stationarity Computation
def compute_residuals_stationarity(Q, q, A, b, C, d, x, s, lamda, mu):
    eq_res = np.linalg.norm(A @ x - b, np.inf)
    in_res = np.linalg.norm(C @ x + s - d, np.inf)
    stat = np.linalg.norm(Q @ x + q + A.T @ lamda + C.T @ mu, np.inf)

    return eq_res, in_res, stat

In [ ]:
# Primal-Dual Augmented Lagrangian Method (problem with equality and inequality constraints only)
def pd_ALM(Q, q, A, b, C, d, x, rho = 0.1, eps_feas = 1e-6, eps_stat = 1e-6):    
    s = projection_nonnegative(d - C @ x) # s0
    lamda = np.zeros(p).reshape(p,1)      # λ0
    mu = np.zeros(m).reshape(m,1)         # μ0
    tau = 10.                             # τ0
        
    best_feas = np.inf
    max_iterations = 200
    counter = 0

    for i in range(max_iterations):
        # (x,s) update
        s = projection_nonnegative(d - C @ x - (1/rho) * mu)
        
        lhs_matrix = Q + rho * (A.T @ A) + rho * (C.T @ C)
        rhs_matrix = - q - A.T @ (lamda - rho * b) - C.T @ (mu - rho * (d - s))
        x = np.linalg.solve(lhs_matrix, rhs_matrix)

        # (λ,μ) update
        lamda += rho * (A @ x - b)
        mu = projection_nonnegative(mu + rho * (C @ x + s - d))

        # Residuals and Stationarity Computation
        eq_res, in_res, stat = compute_residuals_stationarity(Q, q, A, b, C, d, x, s, lamda, mu)
        
        if((eq_res < eps_feas) and (in_res < eps_feas) and (stat < eps_stat)):
            break

        feas = max(eq_res, in_res)
        if(feas < best_feas):
            best_feas = feas
            counter = 0
        else:
            counter += 1

        if (counter >= 2):
            rho *= tau  # increase penalty
        
    return x, s

In [ ]:
def plot_qp(Q, q, A, b, C, d, x, title='', idx=(0, 1), xlim=(-15, 15), ylim=(-15, 15)):
    plt.close()
    fig = plt.figure()
    ax = fig.add_subplot(111)

    N = 200
    x1 = np.linspace(xlim[0], xlim[1], N)
    x2 = np.linspace(ylim[0], ylim[1], N)
    X, Y = np.meshgrid(x1, x2)

    val = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            xx = np.zeros((Q.shape[0], 1))
            xx[idx[0]] = X[i, j]
            xx[idx[1]] = Y[i, j]
            val[i, j] = (f(xx)).item()

    ax.contour(x1.reshape((N,)), x2.reshape((N,)), val, levels=30, cmap='viridis')

    # Plot equality constraints: Ax - b = 0
    if ((A is not None) and (b is not None) and (A.shape[0] > 0)):
        for i in range(A.shape[0]):
            a1 = A[i, idx[0]]
            a2 = A[i, idx[1]]
            bi = b[i]
            if abs(a2) > 1e-12:
                y_eq = (bi - a1 * x1) / a2
                ax.plot(x1, y_eq, 'r-', label=f'Equality constr. ({i+1})')
            else:
                x_eq = bi / a1
                ax.axvline(x_eq, color='r', linestyle='-', label=f'Equality constr. ({i+1})')

    if x is not None:
        ax.plot(x[idx[0]], x[idx[1]], 'rx')

    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.grid(True)
    ax.legend(loc='best')
    ax.set_title(title)
    plt.savefig(title+'.png', dpi=300)
    plt.show()

In [ ]:
q = q.reshape(n,1)
b = b.reshape(p,1)
d = d.reshape(m,1)
x0 = np.random.randn(n).reshape(n,1)

x, s = pd_ALM(Q, q, A, b, C, d, x0)
plot_qp(Q, q, A, b, C, d, x, 'Primal-Dual ALM Algorithm')
plot_qp(Q, q, A, b, C, d, qp.results.x, 'proxqp Solver')

print("---------------------------------------------------------------------------------------------")
print("\"proxqp\" Solver")
print(f"f(x) = {f(qp.results.x.T)[0]}")
print(f"Optimal x = {qp.results.x}")
print(f"Equality constraint: Ax - b = {(A @ qp.results.x - b)[0]}")
print("---------------------------------------------------------------------------------------------")
print("\"pd_ALM()\" Function")
print(f"f(x) = {f(x)[0,0]}")
print(f"Optimal x = {(x.T).flatten()}")
print(f"Equality constraint  : Ax - b = {(A @ x - b)[0]}")
print(f"Inequality constraint: Cx + s - d = {((C @ x + s - d).T)[0]}")
print("---------------------------------------------------------------------------------------------")